Importing the dependcies

In [112]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


Data Collection and Analysis

In [113]:
#loading the dataset from the file
parkinsons_dataset = pd.read_csv('../Datasets/parkinsons.csv')
parkinsons_dataset.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [114]:
# Separate the dataset into two groups based on the 'target' column
outcome_0 = parkinsons_dataset[parkinsons_dataset['status'] == 0]
outcome_1 = parkinsons_dataset[parkinsons_dataset['status'] == 1]

# Undersample the groups
outcome_0_undersampled = outcome_0.sample(n=48, random_state=42)
outcome_1_undersampled = outcome_1.sample(n=50, random_state=42)

# Combine the undersampled groups into a new DataFrame
undersampled_parkinsons_dataset = pd.concat([outcome_0_undersampled, outcome_1_undersampled])

# Shuffle the combined DataFrame
undersampled_parkinsons_dataset = undersampled_parkinsons_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the new DataFrame
print(undersampled_parkinsons_dataset['status'].value_counts())

status
1    50
0    48
Name: count, dtype: int64


In [115]:
print(undersampled_parkinsons_dataset.shape)
undersampled_parkinsons_dataset.info()

(98, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              98 non-null     object 
 1   MDVP:Fo(Hz)       98 non-null     float64
 2   MDVP:Fhi(Hz)      98 non-null     float64
 3   MDVP:Flo(Hz)      98 non-null     float64
 4   MDVP:Jitter(%)    98 non-null     float64
 5   MDVP:Jitter(Abs)  98 non-null     float64
 6   MDVP:RAP          98 non-null     float64
 7   MDVP:PPQ          98 non-null     float64
 8   Jitter:DDP        98 non-null     float64
 9   MDVP:Shimmer      98 non-null     float64
 10  MDVP:Shimmer(dB)  98 non-null     float64
 11  Shimmer:APQ3      98 non-null     float64
 12  Shimmer:APQ5      98 non-null     float64
 13  MDVP:APQ          98 non-null     float64
 14  Shimmer:DDA       98 non-null     float64
 15  NHR               98 non-null     float64
 16  HNR               98 non-null     flo

In [116]:
#checking for missing values
undersampled_parkinsons_dataset.isnull().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

In [117]:
undersampled_parkinsons_dataset.describe()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,161.362173,203.992582,125.037255,0.005745,0.000040,0.003050,0.003123,0.009151,0.026853,0.256102,...,0.042565,0.023028,22.668827,0.510204,0.484147,0.709985,-5.992612,0.210612,2.334003,0.181647
std,47.456160,95.571354,50.499030,0.005003,0.000037,0.003086,0.002730,0.009258,0.019357,0.201599,...,0.030954,0.042318,4.473249,0.502466,0.104683,0.053874,1.154261,0.083703,0.386099,0.091711
min,88.333000,108.664000,67.021000,0.001680,0.000007,0.000680,0.000920,0.002040,0.009540,0.085000,...,0.014030,0.000650,8.441000,0.000000,0.256570,0.626710,-7.964984,0.006274,1.423287,0.044539
25%,118.230000,135.814750,86.380750,0.003058,0.000020,0.001540,0.001672,0.004613,0.015750,0.138000,...,0.023922,0.004783,19.649500,0.000000,0.409772,0.664002,-6.923361,0.160021,2.055479,0.112842
50%,150.822000,191.159500,106.185500,0.004410,0.000030,0.002080,0.002330,0.006240,0.019815,0.187500,...,0.030910,0.009935,23.153500,1.000000,0.464730,0.714422,-6.151102,0.206163,2.318953,0.163058
75%,200.905000,238.613750,148.324750,0.006395,0.000050,0.003445,0.003390,0.010330,0.029825,0.270750,...,0.046878,0.021397,25.526750,1.000000,0.566756,0.756646,-5.437111,0.262358,2.559186,0.224747
max,260.105000,592.030000,239.170000,0.033160,0.000260,0.021440,0.019580,0.064330,0.119080,1.302000,...,0.169420,0.314820,33.047000,1.000000,0.685151,0.823484,-2.434031,0.450493,3.317586,0.527367


In [118]:
#distribution of target variable
undersampled_parkinsons_dataset['status'].value_counts()

status
1    50
0    48
Name: count, dtype: int64

In [119]:
# Select only numeric columns
numeric_columns = undersampled_parkinsons_dataset.select_dtypes(include=[np.number])

# Group by 'status' and calculate the mean
numeric_columns.groupby(undersampled_parkinsons_dataset['status']).mean()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
status,,,,,,,,,,,,,,,,,,,,,
0,181.937771,223.63675,145.207292,0.003866,0.000023,0.001925,0.002056,0.005776,0.017615,0.162958,...,0.028511,0.011483,24.67875,0.0,0.442552,0.695716,-6.759264,0.160292,2.154491,0.123017
1,141.609600,185.13418,105.674020,0.007550,0.000056,0.004130,0.004147,0.012390,0.035721,0.345520,...,0.056056,0.034112,20.73930,1.0,0.524079,0.723684,-5.256627,0.258920,2.506334,0.237931


Seperating the features and target

In [120]:
x = undersampled_parkinsons_dataset.drop(columns=['name','status'], axis=1)
y = undersampled_parkinsons_dataset['status']

In [121]:
x.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,126.144,154.284,97.543,0.00975,0.00008,0.00593,0.00454,0.01778,0.02852,0.266,...,0.02157,0.04499,0.03828,21.534,0.635015,0.627337,-5.070096,0.280091,2.892300,0.249703
1,209.144,237.494,109.379,0.00282,0.00001,0.00147,0.00152,0.00442,0.01861,0.170,...,0.01382,0.02925,0.00871,25.554,0.341788,0.678874,-7.040508,0.066994,2.460791,0.101516
2,102.273,142.830,85.902,0.00907,0.00009,0.00493,0.00461,0.01480,0.02814,0.272,...,0.02073,0.04736,0.03882,18.447,0.671378,0.674562,-2.929379,0.396746,2.560422,0.367233
3,126.344,134.231,112.773,0.00448,0.00004,0.00131,0.00169,0.00393,0.02033,0.185,...,0.01614,0.03429,0.00474,25.030,0.507504,0.760361,-6.689151,0.291954,2.431854,0.105993
4,146.845,208.701,81.737,0.00496,0.00003,0.00250,0.00275,0.00749,0.01919,0.198,...,0.01826,0.02650,0.01328,25.119,0.358773,0.726652,-6.271690,0.196102,2.314209,0.162999


In [122]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: status, dtype: int64

Spliting the data to training data and testing data

In [123]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [124]:
print(x.shape, x_train.shape, x_test.shape)

(98, 22) (78, 22) (20, 22)


Model Training 
Support Vector Machine Learning Models

In [125]:
# model = svm.SVC(kernel='linear')
# model = LogisticRegression()
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [126]:
#training the SVM model with training data
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

Model Evaluation and Accuracy Score

In [127]:
#accuracy score on training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  1.0


In [128]:
#accuracy score on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.95


Building a Predective system

In [129]:
input_data = (197.07600, 206.89600, 192.05500, 0.00289, 0.00001, 0.00280, 0.00270, 0.01309, 0.00006, 0.01309, 0.01309, 0.13200, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309)
# changing input data to numpy array
input_data_as_numpy_array = np.asarray(input_data)
# reshape the array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)
prediction = model.predict(input_data_reshaped)
print(prediction)

if (prediction[0]==1):
  print('The person has Parkinsons Disease')
else:
  print('The person does not have Parkinsons Disease')

[0]
The person does not have Parkinsons Disease


/Users/sakethramansrimattirumala/Desktop/GITAM/Third Year/SEM 6/Personal/AI project version 2_undersample/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Saving the Trained model

In [130]:
import pickle
# Save the model as a pickle in a file
pickle.dump(model, open('parkinsons.sav', 'wb'))

In [131]:
# Load the model from the file
loaded_model = pickle.load(open('parkinsons.sav', 'rb'))

In [132]:
input_data = (197.07600, 206.89600, 192.05500, 0.00289, 0.00001, 0.00280, 0.00270, 0.01309, 0.00006, 0.01309, 0.01309, 0.13200, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309, 0.01309)
# changing input data to numpy array
input_data_as_numpy_array = np.asarray(input_data)
# reshape the array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)
prediction = loaded_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0]==1):
  print('The person has Parkinsons Disease')
else:
  print('The person does not have Parkinsons Disease')

[0]
The person does not have Parkinsons Disease


/Users/sakethramansrimattirumala/Desktop/GITAM/Third Year/SEM 6/Personal/AI project version 2_undersample/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [133]:
for column in x.columns:
    print(column)

MDVP:Fo(Hz)
MDVP:Fhi(Hz)
MDVP:Flo(Hz)
MDVP:Jitter(%)
MDVP:Jitter(Abs)
MDVP:RAP
MDVP:PPQ
Jitter:DDP
MDVP:Shimmer
MDVP:Shimmer(dB)
Shimmer:APQ3
Shimmer:APQ5
MDVP:APQ
Shimmer:DDA
NHR
HNR
RPDE
DFA
spread1
spread2
D2
PPE
